In [7]:
import warnings
import os
import pandas as pd

# Supprime les avertissements de dépréciation de numpy/tensorflow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
warnings.filterwarnings('ignore', category=FutureWarning)

# Configuration de l'affichage pour Notebook
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [16]:

from mh_nlp.domain.entities.document import Document
from mh_nlp.domain.services.text_cleaner import TextCleaner

# ==========================================
# 1. INITIALISATION DU SERVICE
# ==========================================
# Instanciation du cleaner (Logique métier du domaine)
cleaner = TextCleaner()

# ==========================================
# 2. PRÉPARATION DU JEU DE TEST (DATASET)
# ==========================================
# Liste exhaustive couvrant les cas limites (Edge Cases)
data = [
    # --- Groupe A : Cas Standards & Émotions ---
    Document(text="I am NOT feeling good today... 😞"),
    Document(text="I feel anxious and I can't sleep."),
    Document(text="I am feeling very happy and energetic!"),
    Document(text="I feel very depressed and lonely today."),
    Document(text="vErY ANXIOUS AND sTrEsSeD out right now."),
    Document(text="Sad."),
    
    # --- Groupe B : Bruit Social Media & Web ---
    Document(text="Check this out: https://help.com/support @user123 #mentalhealth"),
    Document(text="RT @user123: This is so sad!!! #mentalhealth #depression #help @charity_org"),
    Document(text="Found help here: https://support.org/help?id=123 and http://test.com/path"),
    Document(text="<p>This is a <b>test</b> of HTML removal.</p>"),
    
    # --- Groupe C : Ponctuation & Caractères Spéciaux ---
    Document(text="HELP ME !!!... ??? (I am not okay) [urgent] *sigh*"),
    Document(text="!!! ??? @@@ ###"),
    Document(text="I am happy today! 😊🌟 Love life! ❤️"),
    Document(text="IsThisReal?NoItIsnt!HelpMe."),
    
    # --- Groupe D : Formats Techniques & Nombres ---
    Document(text="I have been suffering since 2010. 10/10 would not recommend."),
    Document(text="First line.\nSecond line with\ttabs."),
    Document(text="1234567890"),
    Document(text="2023-01-01 12:00:00 [ERROR] User is feeling low"),
    
    # --- Groupe E : Cas "Vides" ou Problématiques ---
    Document(text="    "), 
    Document(text=" "),
    Document(text=""),
    Document(text="/ restlessness appears. Thanks!"),
    Document(text="I am " + "so " * 50 + "tired."), # Test de performance/longueur
    
    # --- Groupe F : Linguistique (Apostrophes & Slang) ---
    Document(text="I can't sleep, I'm exhausted and shouldn't stay up."),
    Document(text="I am soooooooo hhaapppyyyyyyy")
]

# ==========================================
# 3. EXÉCUTION DU PIPELINE DE NETTOYAGE
# ==========================================
results = []

for doc in data:
    # Appel de la méthode clean du domaine
    cleaned_text = cleaner.clean(doc)
    
    results.append({
        "Original": doc.text.strip()[:100] + "..." if len(doc.text) > 100 else doc.text,
        "Cleaned": cleaned_text,
        "Length_Before": len(doc.text),
        "Length_After": len(cleaned_text),
        "Status": "OK" if cleaned_text else "Empty"
    })

# ==========================================
# 4. VISUALISATION ET ANALYSE DES RÉSULTATS
# ==========================================
df_cleaned = pd.DataFrame(results)

print(f"Analyse de {len(data)} documents terminée.\n")
display(df_cleaned)

2026-01-17 01:39:49.952 | WARNING  | mh_nlp.domain.services.text_cleaner:clean:38 - Document vide ou nul reçu → nettoyage ignoré.
2026-01-17 01:39:49.954 | WARNING  | mh_nlp.domain.services.text_cleaner:clean:38 - Document vide ou nul reçu → nettoyage ignoré.
2026-01-17 01:39:49.954 | WARNING  | mh_nlp.domain.services.text_cleaner:clean:38 - Document vide ou nul reçu → nettoyage ignoré.


Analyse de 25 documents terminée.



,Original,Cleaned,Length_Before,Length_After,Status
0,I am NOT feeling good today... 😞,am not feeling good today,32,25,OK
1,I feel anxious and I can't sleep.,feel anxious and can't sleep,33,28,OK
2,I am feeling very happy and energetic!,am feeling very happy and energetic,38,35,OK
3,I feel very depressed and lonely today.,feel very depressed and lonely today,39,36,OK
4,vErY ANXIOUS AND sTrEsSeD out right now.,very anxious and stressed out right now,40,39,OK
5,Sad.,sad,4,3,OK
6,Check this out: https://help.com/support @user123 #mentalhealth,check this out,63,14,OK
7,RT @user123: This is so sad!!! #mentalhealth #depression #help @charity_org,rt this is so sad,75,17,OK
8,Found help here: https://support.org/help?id=123 and http://test.com/path,found help here and,73,19,OK
9,<p>This is a <b>test</b> of HTML removal.</p>,this is test of html removal,45,28,OK


In [17]:

from mh_nlp.infrastructure.nlp.spacy_tokenizer import SpacyTokenizer

documents = [Document(text) for text in df_cleaned.Cleaned]

spacy_tok = SpacyTokenizer(model_name="en_core_web_sm", lemmatize=True, remove_stop= True, batch_size=10)
spacy_output = spacy_tok.tokenize(documents)

for i in range(len(documents)):
    print(f"Text nettoyé original: {documents[i].text}")
    print(f"Sortie spaCy (Lemmes): {spacy_output[i]}")
    print("--"*3)

    if i > 5:
        break


2026-01-17 01:39:50.408 | INFO     | mh_nlp.infrastructure.nlp.spacy_tokenizer:__init__:40 - Initialisation de SpacyTokenizer : en_core_web_sm
2026-01-17 01:39:50.810 | SUCCESS  | mh_nlp.infrastructure.nlp.spacy_tokenizer:__init__:43 - Modèle spaCy 'en_core_web_sm' prêt.
2026-01-17 01:39:50.811 | INFO     | mh_nlp.infrastructure.nlp.spacy_tokenizer:tokenize:64 - Tokenisation de 25 documents (Batch: 10)
NLP Processing: 100%|██████████| 25/25 [00:00<00:00, 700.87doc/s]

Text nettoyé original: am not feeling good today
Sortie spaCy (Lemmes): ['feel', 'good', 'today']
------
Text nettoyé original: feel anxious and can't sleep
Sortie spaCy (Lemmes): ['feel', 'anxious', 'sleep']
------
Text nettoyé original: am feeling very happy and energetic
Sortie spaCy (Lemmes): ['feel', 'happy', 'energetic']
------
Text nettoyé original: feel very depressed and lonely today
Sortie spaCy (Lemmes): ['feel', 'depressed', 'lonely', 'today']
------
Text nettoyé original: very anxious and stressed out right now
Sortie spaCy (Lemmes): ['anxious', 'stress', 'right']
------
Text nettoyé original: sad
Sortie spaCy (Lemmes): ['sad']
------
Text nettoyé original: check this out
Sortie spaCy (Lemmes): ['check']
------


In [18]:
from mh_nlp.infrastructure.nlp.spacy_tokenizer import SpacyTokenizer

documents = [Document(text) for text in df_cleaned.Cleaned]

spacy_tok = SpacyTokenizer(model_name="en_core_web_sm", lemmatize=True, batch_size=10)
spacy_output = spacy_tok.tokenize(documents)

for i in range(len(documents)):
    print(f"Text nettoyé original: {documents[i].text}")
    print(f"Sortie spaCy (Lemmes): {spacy_output[i]}")
    print("--"*3)

    if i > 5:
        break


2026-01-17 01:39:51.062 | INFO     | mh_nlp.infrastructure.nlp.spacy_tokenizer:__init__:40 - Initialisation de SpacyTokenizer : en_core_web_sm
2026-01-17 01:39:51.329 | SUCCESS  | mh_nlp.infrastructure.nlp.spacy_tokenizer:__init__:43 - Modèle spaCy 'en_core_web_sm' prêt.
2026-01-17 01:39:51.330 | INFO     | mh_nlp.infrastructure.nlp.spacy_tokenizer:tokenize:64 - Tokenisation de 25 documents (Batch: 10)
NLP Processing: 100%|██████████| 25/25 [00:00<00:00, 1289.60doc/s]

Text nettoyé original: am not feeling good today
Sortie spaCy (Lemmes): ['feel', 'good', 'today']
------
Text nettoyé original: feel anxious and can't sleep
Sortie spaCy (Lemmes): ['feel', 'anxious', 'sleep']
------
Text nettoyé original: am feeling very happy and energetic
Sortie spaCy (Lemmes): ['feel', 'happy', 'energetic']
------
Text nettoyé original: feel very depressed and lonely today
Sortie spaCy (Lemmes): ['feel', 'depressed', 'lonely', 'today']
------
Text nettoyé original: very anxious and stressed out right now
Sortie spaCy (Lemmes): ['anxious', 'stress', 'right']
------
Text nettoyé original: sad
Sortie spaCy (Lemmes): ['sad']
------
Text nettoyé original: check this out
Sortie spaCy (Lemmes): ['check']
------


In [20]:
from loguru import logger
from mh_nlp.domain.services.text_cleaner import TextCleaner
from mh_nlp.infrastructure.data.kaggle_repository import KaggleDatasetRepository
from mh_nlp.application.use_cases.build_dataset import BuildCleanDatasetUseCase

# --- 1. CONFIGURATION ---
label_mapping = {
    "Anxiety": 0, 
    "Normal": 1, 
    "Depression": 2,
}

# --- 2. INSTANCIATION DES COMPOSANTS ---
# Initialisation de l'accès aux données et du service de nettoyage
repository = KaggleDatasetRepository(
    csv_path="../data/raw/mental_health.csv",
    label_mapping=label_mapping,
)

cleaner = TextCleaner()

# --- 3. ORCHESTRATION ---
# Injection des dépendances dans le cas d'usage
use_case = BuildCleanDatasetUseCase(
    repository=repository, 
    cleaner=cleaner
)

# --- 4. EXÉCUTION ---
# Lancement du pipeline et récupération du DatasetDTO
dataset_final = use_case.execute()

# --- 5. Extraction des données (List comprehension pour plus d'efficacité)
texts = [doc.text for doc in dataset_final.documents]
labels = [label for label in dataset_final.labels]

# Création du DataFrame
# Utiliser un dictionnaire est la méthode la plus propre
dataset = pd.DataFrame({
    'cleaned text': texts,
    'label': labels,
})

# Afficher les premières lignes pour vérifier
display(dataset.head(3))

2026-01-17 01:40:48.001 | INFO     | mh_nlp.application.use_cases.build_dataset:execute:47 - Démarrage du pipeline BuildCleanDataset.
2026-01-17 01:40:48.002 | INFO     | mh_nlp.infrastructure.data.kaggle_repository:load:33 - Démarrage du chargement des données depuis : ../data/raw/mental_health.csv
2026-01-17 01:40:48.299 | SUCCESS  | mh_nlp.infrastructure.data.kaggle_repository:load:57 - Infrastructure : 1000 paires (Document, Label) créées.
2026-01-17 01:40:48.302 | DEBUG    | mh_nlp.application.use_cases.build_dataset:execute:66 - Traitement unitaire de 1000 documents.
2026-01-17 01:40:48.337 | SUCCESS  | mh_nlp.application.use_cases.build_dataset:execute:92 - Pipeline terminé : 1000 entités synchronisées. (0 supprimées car vides).


,cleaned text,label
0,oh my gosh,0
1,trouble sleeping confused mind restless heart all out of tune,0
2,all wrong back off dear forward doubt stay in restless and restless place,0


# Split Clened Dataset

In [22]:
from mh_nlp.application.use_cases.split_dataset import SplitDatasetUseCase

# 1. Instanciation du Use Case avec vos ratios
# Ici: 70% Train / 10% Val / 20% Test
splitter = SplitDatasetUseCase(test_size=0.2, val_size=0.1)

# 2. Exécution du split
splits = splitter.execute(dataset_final)

# 3. Récupération des résultats
train_set = splits["train"]
val_set = splits["val"]
test_set = splits["test"]

print(f"Taille Train : {train_set.total_records}")
print(f"Premier document Train : {train_set.documents[0].text}")
print(f"Premier label Train : {train_set.labels[0]}")

2026-01-17 01:43:44.783 | INFO     | mh_nlp.application.use_cases.split_dataset:execute:53 - Découpage du dataset (Total: 1000) | Cible: Test=0.2, Val=0.1
2026-01-17 01:43:44.841 | SUCCESS  | mh_nlp.application.use_cases.split_dataset:execute:86 - Split réussi : Train=700 | Val=100 | Test=200


Taille Train : 700
Premier document Train : anxiety fear overthinking at the same time can be called panic attack right
Premier label Train : 0


# Apply Tokenizer fonction

In [24]:
from mh_nlp.domain.entities.document import Document

# Jeu de test minimaliste
raw_texts = [
    "I am feeling very anxious today and I can't sleep.",
    "The therapy sessions are helping me manage my stress better.",
    "I feel much better after talking to my friend."
]

# Conversion en entités du domaine
documents = [Document(text) for text in raw_texts]

# A. Le Tokenizer spaCy (Linguistique)

In [25]:
from mh_nlp.infrastructure.nlp.spacy_tokenizer import SpacyTokenizer

spacy_tok = SpacyTokenizer(model_name="en_core_web_sm", lemmatize=True, batch_size=2)
spacy_output = spacy_tok.tokenize(documents)

print("--- Sortie spaCy (Lemmes) ---")
print(spacy_output[0]) 
# Devrait afficher: ['feel', 'anxious', 'today', 'sleep']

2026-01-17 13:59:13.025 | INFO     | mh_nlp.infrastructure.nlp.spacy_tokenizer:__init__:40 - Initialisation de SpacyTokenizer : en_core_web_sm
2026-01-17 13:59:15.213 | SUCCESS  | mh_nlp.infrastructure.nlp.spacy_tokenizer:__init__:43 - Modèle spaCy 'en_core_web_sm' prêt.
2026-01-17 13:59:15.234 | INFO     | mh_nlp.infrastructure.nlp.spacy_tokenizer:tokenize:64 - Tokenisation de 3 documents (Batch: 2)
NLP Processing: 100%|██████████| 3/3 [00:00<00:00, 28.39doc/s]

--- Sortie spaCy (Lemmes) ---
['feel', 'anxious', 'today', 'sleep']


# B. Le Tokenizer HuggingFace (Deep Learning)

In [26]:
from mh_nlp.infrastructure.nlp.hf_tokenizer import HuggingFaceTokenizer

hf_tok = HuggingFaceTokenizer(model_name="distilbert-base-uncased", max_length=16)
hf_output = hf_tok.tokenize(documents)

print("\n--- Sortie HuggingFace (Tenseurs) ---")
print(hf_output['input_ids'][0]) # Affiche les IDs numériques
print(f"Shape: {hf_output['input_ids'].shape}")

2026-01-17 13:59:22.548 | INFO     | mh_nlp.infrastructure.nlp.hf_tokenizer:__init__:34 - Chargement du tokenizer HF : distilbert-base-uncased (max_length=16)
2026-01-17 13:59:23.703 | SUCCESS  | mh_nlp.infrastructure.nlp.hf_tokenizer:__init__:36 - Tokenizer distilbert-base-uncased chargé avec succès.
2026-01-17 13:59:23.708 | DEBUG    | mh_nlp.infrastructure.nlp.hf_tokenizer:tokenize:63 - Tokenisation en cours pour 3 documents (Modèle: distilbert-base-uncased)
2026-01-17 13:59:23.740 | DEBUG    | mh_nlp.infrastructure.nlp.hf_tokenizer:tokenize:76 - Tokenisation terminée. Forme du tenseur : [3, 15]



--- Sortie HuggingFace (Tenseurs) ---
tensor([  101,  1045,  2572,  3110,  2200, 11480,  2651,  1998,  1045,  2064,
         1005,  1056,  3637,  1012,   102])
Shape: torch.Size([3, 15])


In [27]:
hf_output['input_ids']

tensor([[  101,  1045,  2572,  3110,  2200, 11480,  2651,  1998,  1045,  2064,
          1005,  1056,  3637,  1012,   102],
        [  101,  1996,  7242,  6521,  2024,  5094,  2033,  6133,  2026,  6911,
          2488,  1012,   102,     0,     0],
        [  101,  1045,  2514,  2172,  2488,  2044,  3331,  2000,  2026,  2767,
          1012,   102,     0,     0,     0]])

# C. Le Tokenizer Keras (CNN/LSTM)

In [16]:
from mh_nlp.infrastructure.nlp.keras_tokenizer import KerasTextTokenizer

keras_tok = KerasTextTokenizer(vocab_size=1000, max_length=16)
keras_tok.fit(documents) # Obligatoire pour Keras
keras_output = keras_tok.tokenize(documents)

print("\n--- Sortie Keras (Numpy Matrix) ---")
print(keras_output[0])
print(f"Shape: {keras_output.shape}")

2026-01-16 11:34:06.066 | INFO     | mh_nlp.infrastructure.nlp.keras_tokenizer:__init__:30 - KerasTextTokenizer initialisé (Vocab target: 1000, Max length: 16)
2026-01-16 11:34:06.079 | INFO     | mh_nlp.infrastructure.nlp.keras_tokenizer:fit:46 - Apprentissage du vocabulaire sur 3 documents...
2026-01-16 11:34:06.082 | SUCCESS  | mh_nlp.infrastructure.nlp.keras_tokenizer:fit:52 - Vocabulaire appris : 26 mots uniques trouvés.
2026-01-16 11:34:06.093 | DEBUG    | mh_nlp.infrastructure.nlp.keras_tokenizer:tokenize:82 - Séquences générées : (3, 16) (Type: int32)



--- Sortie Keras (Numpy Matrix) ---
[ 2  5  6  7  8  9 10  2 11 12  0  0  0  0  0  0]
Shape: (3, 16)
